In [14]:
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV()
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [21]:
df = pd.read_csv('polydf.csv',index_col='Unnamed: 0')
df1 = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')
df


,Age Nscore Oscore,Age Nscore Ascore,Age Escore Oscore,Age Escore Ascore,Age Escore Cscore,Age Oscore Ascore,Age Oscore SensationSeeking,Age Ascore Cscore,Education Nscore Oscore,Education Nscore Ascore,Education Nscore Cscore,Education Escore Oscore,Education Escore Cscore,Education Ascore Cscore,Nscore Escore Impulsive,Nscore Escore SensationSeeking,Nscore Escore Nicotine,Nscore Oscore SensationSeeking,Nscore Oscore Chocalate,Nscore Oscore Nicotine,Nscore Ascore Impulsive,Nscore Ascore SensationSeeking,Nscore Ascore Caff,Nscore Ascore Chocalate,Nscore Ascore Nicotine,Escore Oscore Impulsive,Escore Oscore SensationSeeking,Escore Oscore Caff,Escore Ascore Cscore,Escore Ascore SensationSeeking,Escore Cscore Chocalate,Escore Cscore Nicotine,Oscore Ascore Cscore,Oscore Ascore Impulsive,Oscore Ascore SensationSeeking,Oscore Ascore Chocalate,Oscore Ascore Nicotine,Oscore Cscore Nicotine,Oscore Impulsive SensationSeeking,Ascore Cscore Impulsive,Ascore Cscore SensationSeeking,Ascore Cscore Caff,Ascore Cscore Nicotine
0,4914.0,4329.0,4536.0,3996.0,4536.0,4662.0,378.0,4662.0,1638.0,1443.0,1638.0,1512.0,1512.0,1554.0,5616.0,4212.0,2808.0,4914.0,8190.0,3276.0,5772.0,4329.0,8658.0,7215.0,2886.0,6048.0,4536.0,9072.0,55944.0,3996.0,7560.0,3024.0,65268.0,6216.0,4662.0,7770.0,3108.0,3528.0,504.0,6216.0,4662.0,9324.0,3108.0
1,3190.0,2784.0,5720.0,4992.0,4264.0,5280.0,660.0,3936.0,6380.0,5568.0,4756.0,11440.0,8528.0,7872.0,4524.0,9048.0,6032.0,9570.0,9570.0,6380.0,4176.0,8352.0,8352.0,8352.0,5568.0,8580.0,17160.0,17160.0,102336.0,14976.0,12792.0,8528.0,108240.0,7920.0,15840.0,15840.0,10560.0,9020.0,990.0,5904.0,11808.0,11808.0,7872.0
2,3720.0,2976.0,5400.0,4320.0,4590.0,3840.0,960.0,3264.0,1240.0,992.0,1054.0,1800.0,1530.0,1088.0,2790.0,11160.0,0.0,9920.0,4960.0,0.0,1984.0,7936.0,5952.0,3968.0,0.0,3600.0,14400.0,10800.0,48960.0,11520.0,6120.0,0.0,43520.0,2560.0,10240.0,5120.0,0.0,0.0,640.0,2176.0,8704.0,6528.0,0.0
3,1564.0,1598.0,1564.0,1598.0,1564.0,2162.0,138.0,2162.0,4692.0,4794.0,4692.0,4692.0,4692.0,6486.0,2312.0,3468.0,2312.0,4692.0,6256.0,3128.0,3196.0,4794.0,7990.0,6392.0,3196.0,3128.0,4692.0,7820.0,73508.0,4794.0,6256.0,3128.0,99452.0,4324.0,6486.0,8648.0,4324.0,4232.0,276.0,4324.0,6486.0,10810.0,4324.0
4,5547.0,5289.0,3612.0,3444.0,4200.0,5289.0,774.0,6150.0,7396.0,7052.0,8600.0,4816.0,5600.0,8200.0,4816.0,7224.0,2408.0,11094.0,11094.0,3698.0,7052.0,10578.0,10578.0,10578.0,3526.0,4816.0,7224.0,7224.0,57400.0,6888.0,8400.0,2800.0,88150.0,7052.0,10578.0,10578.0,3526.0,4300.0,1032.0,8200.0,12300.0,12300.0,4100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1425.0,1200.0,2907.0,2448.0,1683.0,2736.0,627.0,1584.0,0.0,0.0,0.0,0.0,0.0,0.0,8925.0,14025.0,0.0,15675.0,5700.0,0.0,8400.0,13200.0,4800.0,4800.0,0.0,20349.0,31977.0,11628.0,80784.0,26928.0,6732.0,0.0,90288.0,19152.0,30096.0,10944.0,0.0,0.0,4389.0,11088.0,17424.0,6336.0,0.0
1881,1650.0,1584.0,2550.0,2448.0,1530.0,2400.0,450.0,1440.0,0.0,0.0,0.0,0.0,0.0,0.0,11781.0,15147.0,8415.0,14850.0,6600.0,8250.0,11088.0,14256.0,7920.0,6336.0,7920.0,17850.0,22950.0,12750.0,73440.0,22032.0,6120.0,7650.0,72000.0,16800.0,21600.0,9600.0,12000.0,7500.0,3150.0,10080.0,12960.0,7200.0,7200.0
1882,3478.0,2914.0,2220.0,1860.0,1860.0,2294.0,370.0,1922.0,3478.0,2914.0,2914.0,2220.0,1860.0,1922.0,8460.0,7050.0,8460.0,8695.0,10434.0,10434.0,8742.0,7285.0,8742.0,8742.0,8742.0,6660.0,5550.0,6660.0,28830.0,4650.0,5580.0,5580.0,35557.0,6882.0,5735.0,6882.0,6882.0,6882.0,1110.0,5766.0,4805.0,5766.0,5766.0
1883,2160.0,1440.0,1248.0,832.0,572.0,1536.0,480.0,704.0,0.0,0.0,0.0,0.0,0.0,0.0,9360.0,11700.0,4680.0,21600.0,10800.0,8640.0,11520.0,14400.0,8640.0,7200.0,5760.0,9984.0,12480.0,7488.0,18304.0,8320.0,2860.0,2288.0,33792.0,12288.0,15360.0,7680.0,6144.0,4224.0,3840.0,5632.0,7040.0,4224.0,2816.0


In [22]:
stimtestdata = df1.drop(['hallucinagen','depressant'],axis = 1)
stimx = df
stimy = stimtestdata.stimulant
stimy


1        low
2        low
3       None
4        low
5        low
        ... 
1884    None
1885    None
1886    high
1887    None
1888     low
Name: stimulant, Length: 1885, dtype: object

In [23]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)

In [24]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

1571    None
798      low
1817    None
1360    high
1108    None
        ... 
418     high
222     None
1446    None
1168     low
1266    None
Name: stimulant, Length: 1413, dtype: object

Logistic regression classifiers

In [27]:
mod = LogisticRegression(class_weight = 'balanced'
                         ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)

In [ ]:
logregparam_grid = {'C': [1, 10], 'solver': ['linear', 'rbf','sag']}
GridSearchCV(Logisticregression, param_grid, n_jobs=-1, cv=5, verbose=1)

In [28]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.6274908768244966, 0.5839927035935507)

In [160]:
mod2 = LogisticRegression(max_iter=4000,solver='liblinear',warm_start=True,tol=1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

In [161]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')

(0.5964628068500198, 0.5664527904280448)

In [144]:
mod3 = LogisticRegression(class_weight='balanced',max_iter=4000,solver='liblinear',tol=1)
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

In [145]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.6223106438016687, 0.5845346852802334)

Decision Trees

In [140]:
dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0055)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

In [141]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')




(0.6251055171091874, 0.5798962793128896)

In [199]:
dtc2 = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.006,class_weight='balanced')
dtc2.fit(stimXtrain,stimYtrain)
dtc2ytrainpred = dtc2.predict(stimXtrain)
dtc2ytestpred = dtc2.predict(stimXtest)

In [200]:
f1_score(stimYtrain,dtc2ytrainpred,average='weighted'),f1_score(stimYtest,dtc2ytestpred,average='weighted')


(0.6143686467532781, 0.5408066496076716)

In [130]:
dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.006599999)
dtc3.fit(stimXtrain,stimYtrain)
dtc3ytrainpred = dtc3.predict(stimXtrain)
dtc3ytestpred = dtc3.predict(stimXtest)

In [131]:
f1_score(stimYtrain,dtc3ytrainpred,average='weighted'),f1_score(stimYtest,dtc3ytestpred,average='weighted')


(0.6151310398457286, 0.5412524778938459)

Knearest

In [202]:
knn = KNeighborsClassifier(n_neighbors=250,weights='uniform')
knn.fit(stimXtrain,stimYtrain)
knntrainypred=knn.predict(stimXtrain)
knntestypred=knn.predict(stimXtest)
f1_score(stimYtrain,knntrainypred,average='weighted'),f1_score(stimYtest,knntestypred,average='weighted')

(0.5797289490801543, 0.5577499100840714)

In [203]:
knn2 = KNeighborsClassifier(n_neighbors=15,weights='uniform')
knn2.fit(stimXtrain,stimYtrain)
knn2trainypred=knn2.predict(stimXtrain)
knn2testypred=knn2.predict(stimXtest)
f1_score(stimYtrain,knn2trainypred,average='weighted'),f1_score(stimYtest,knn2testypred,average='weighted')

(0.6394334409619199, 0.5683806227682628)

Random Forest

In [83]:
rfc = RandomForestClassifier(criterion='gini',ccp_alpha=0.009,n_estimators=1000,max_features=6,class_weight='balanced',max_depth=6,oob_score=True)
rfc.fit(stimXtrain,stimYtrain)

RandomForestClassifier(ccp_alpha=0.009, class_weight='balanced', max_depth=6,
                       max_features=6, n_estimators=1000, oob_score=True)

In [84]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')

(0.5918547315218621, 0.5489259492854196)

In [213]:
rfc2 = RandomForestClassifier(criterion='entropy',ccp_alpha=0.02
                              ,n_estimators=100000,class_weight='balanced'
                              ,oob_score=True,n_jobs=-1,verbose = True,max_features=30)
rfc2.fit(stimXtrain,stimYtrain)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 11226 tasks 

RandomForestClassifier(ccp_alpha=0.02, class_weight='balanced',
                       criterion='entropy', max_features=30,
                       n_estimators=100000, n_jobs=-1, oob_score=True,
                       verbose=True)

In [214]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 1226 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 1776 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 2426 tasks      | elapsed:    0.5s
[Parallel(n_jobs=12)]: Done 3176 tasks      | elapsed:    0.6s
[Parallel(n_jobs=12)]: Done 4026 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 4976 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done 6026 tasks      | elapsed:    1.1s
[Parallel(n_jobs=12)]: Done 7176 tasks      | elapsed:    1.3s
[Parallel(n_jobs=12)]: Done 8426 tasks      | elapsed:    1.6s
[Parallel(n_jobs=12)]: Done 9776 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done 11226 tasks 

(0.6003655853402908, 0.5565019234805496)

voting classifier

In [215]:
vtc = VotingClassifier([('dtc2',dtc2),('rfc2',rfc2),('knn',knn),('knn2',knn2),('log',mod),('log3',mod3),('log2',mod2),('dtc',dtc),('dtc3',dtc3),("rfc",rfc)],voting='soft',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('dtc2',
                              DecisionTreeClassifier(ccp_alpha=0.006,
                                                     class_weight='balanced')),
                             ('rfc2',
                              RandomForestClassifier(ccp_alpha=0.02,
                                                     class_weight='balanced',
                                                     criterion='entropy',
                                                     max_features=30,
                                                     n_estimators=100000,
                                                     n_jobs=-1, oob_score=True,
                                                     verbose=True)),
                             ('knn', KNeighborsClassifier(n_neighbors=250)),
                             ('knn2', KNeighborsClassifier(n_neigh...
                              LogisticRegression(max_iter=4000,
                                                 

In [216]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 1226 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 1776 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 2426 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 3176 tasks      | elapsed:    0.5s
[Parallel(n_jobs=12)]: Done 4026 tasks      | elapsed:    0.6s
[Parallel(n_jobs=12)]: Done 4976 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 6026 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done 7176 tasks      | elapsed:    1.1s
[Parallel(n_jobs=12)]: Done 8426 tasks      | elapsed:    1.3s
[Parallel(n_jobs=12)]: Done 9776 tasks      | elapsed:    1.5s
[Parallel(n_jobs=12)]: Done 11226 tasks 

0.6582121313304148 0.5951090892458898


In [207]:
pkl.dump(vtc,open('vtcpoly.pkl','wb'))